In [ ]:
#| default_exp learner

In [ ]:
#| export 
import math, torch, matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from operator import attrgetter
from functools import partial
from copy import copy

from torch import optim
import torch.nn.functional as F

from miniai.conv import *

from fastprogress import progress_bar, master_bar

In [ ]:
import matplotlib as mpl
import torchvision.transforms.functional as TF
from contextlib import contextmanager
from torch import nn, tensor
from datasets import load_dataset, load_dataset_builder
from miniai.datasets import *
import logging
from fastcore.test import test_close

In [ ]:
torch.set_printoptions(precision=2, linewidth=160, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [ ]:
logging.disable(logging.WARNING)

## Learner

In [ ]:
x, y = 'image', 'label'
name = 'fashion_mnist'
dsd = load_dataset(name)

  0%|          | 0/2 [00:00<?, ?it/s]

Grab a single example from Dataset and check its shape. We will be working with flattened images, so we convert PIL to tensor and flatten it out to get 784 (28x28) long tensor

In [ ]:
ex = TF.to_tensor(dsd['train'][0][x])
ex.shape

torch.Size([1, 28, 28])

In [ ]:
torch.flatten(TF.to_tensor(dsd['train'][0][x])).shape

torch.Size([784])

In [ ]:
@inplace
def transformi(b):
    # b is a dictionary of image and label
    # import ipdb; ipdb.set_trace()
    b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [ ]:
bs = 1024
tds = dsd.with_transform(transformi)

In [ ]:
dls.train??

Type:        DataLoader
String form: <torch.utils.data.dataloader.DataLoader object>
Length:      59
File:        /usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py
Source:     
class DataLoader(Generic[T_co]):
    r"""
    Data loader. Combines a dataset and a sampler, and provides an iterable over
    the given dataset.

    The :class:`~torch.utils.data.DataLoader` supports both map-style and
    iterable-style datasets with single- or multi-process loading, customizing
    loading order and optional automatic batching (collation) and memory pinning.

    See :py:mod:`torch.utils.data` documentation page for more details.

    Args:
        dataset (Dataset): dataset from which to load the data.
        batch_size (int, optional): how many samples per batch to load
            (default: ``1``).
        shuffle (bool, optional): set to ``True`` to have the data reshuffled
            at every epoch (default: ``False``).
        sampler (Sampler or Iterable, optiona

In [ ]:
# remove number of workers during debugging
ipdb.set_trace(); dls = DataLoaders.from_dd(tds, bs)
dt = dls.train
xb, yb = next(iter(dt))
xb.shape, yb[:10]

--Return--
None
> /tmp/ipykernel_1258/3029029536.py(2)<cell line: 2>()
      1 # remove number of workers during debugging
----> 2 ipdb.set_trace(); dls = DataLoaders.from_dd(tds, bs)
      3 dt = dls.train



ipdb>  q


Let's remind ourselves how `DataLoaders.from_dd` actually works. 

1. Create an internal collation function `f` that itself returns a function that returns `x` and `y` values based on the features of a given dataset. This is how it is done:
    1. `collate_dict` takes featutes from `dd['train']`: `image` and `label` and puts them into an itemgetter
    2. applies this itemgetter to the result of a `default_collate` on a given Dataset returning a tupel of `x` and `y` values
2. This collation function is then used to create dataloaders from dataset using get_dls function on values of a dataset. Values of a dataset are 'train' and 'test'. They are wrapped inside the DataLoader itself using `cls`


In [ ]:
def o():
    ipdb.set_trace()
    dls = DataLoaders.from_dd(tds, bs)
    pass
o()

> /tmp/ipykernel_1258/2284370029.py(3)o()
      2     ipdb.set_trace()
----> 3     dls = DataLoaders.from_dd(tds, bs)
      4     pass



ipdb>  s


--Call--
> /notebooks/fastai-p2/miniai/datasets.py(106)from_dd()
    105 
--> 106     @classmethod
    107     def from_dd(cls, dd, batch_size, as_tuple=True, **kwargs):



ipdb>  n


> /notebooks/fastai-p2/miniai/datasets.py(108)from_dd()
    107     def from_dd(cls, dd, batch_size, as_tuple=True, **kwargs):
--> 108         f = collate_dict(dd['train'])
    109         return cls(*get_dls(*dd.values(), bs=batch_size, collate_fn=f, **kwargs))



...

ipdb>  n


> /tmp/ipykernel_1258/2284370029.py(4)o()
      3     dls = DataLoaders.from_dd(tds, bs)
----> 4     pass
      5 o()



ipdb>  n


--Return--
None
> /tmp/ipykernel_1258/2284370029.py(4)o()
      3     dls = DataLoaders.from_dd(tds, bs)
----> 4     pass
      5 o()



ipdb>  c


In [ ]:
tds['train']

Dataset({
    features: ['image', 'label'],
    num_rows: 60000
})

In [ ]:
??collate_dict(tds)

Signature: collate_dict(ds)
Docstring: <no docstring>
Source:   
def collate_dict(ds):
    # import ipdb; ipdb.set_trace()
    get = itemgetter(*ds.features) # get x and y values
    def _f(b): return get(default_collate(b))
    return _f
File:      /notebooks/fastai-p2/miniai/datasets.py
Type:      function


In [ ]:
tds['train'][]

{'image': [tensor([0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
          0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
          0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
          0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05,
          0.29, 0.00, 0.00, 0.00, 0.02, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01,
          0.00, 0.14, 0.53, 0.50, 0.24, 0.21, 0.00, 0.00, 0.00, 0.00, 0.01, 0.02, 0.00, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
          0.00, 0.00, 0.02, 0.0

In [ ]:
collate_dict(tds['train'])(tds['train'][:10])

KeyError: 0

In [ ]:
tds.values()

dict_values([Dataset({
    features: ['image', 'label'],
    num_rows: 60000
}), Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})])

In [ ]:
 DataLoaders.from_dd??

Signature: DataLoaders.from_dd(dd, batch_size, as_tuple=True, **kwargs)
Docstring: <no docstring>
Source:   
    @classmethod
    def from_dd(cls, dd, batch_size, as_tuple=True, **kwargs):
        f = collate_dict(dd['train'])
        return cls(*get_dls(*dd.values(), bs=batch_size, collate_fn=f, **kwargs))
File:      /notebooks/fastai-p2/miniai/datasets.py
Type:      method


When is collate_fn used?????? Inside get_dls!!! Add comments?

In [ ]:
DataLoaders??

Init signature: DataLoaders(*dls)
Docstring:      <no docstring>
Source:        
class DataLoaders: #??????
    def __init__(self, *dls): self.train, self.valid = dls[:2]
    
    @classmethod
    def from_dd(cls, dd, batch_size, as_tuple=True, **kwargs):
        f = collate_dict(dd['train'])
        return cls(*get_dls(*dd.values(), bs=batch_size, collate_fn=f, **kwargs))
File:           /notebooks/fastai-p2/miniai/datasets.py
Type:           type
Subclasses:     


NameError: name 'cls' is not defined